In [ ]:
#Part2-function2

import math

def checkinit(s_x0, s_y0, v_x0, v_y0, Rp):
    
    """
    checks whether the initial spacecraft conditions are valid
    
    conditions:
    
    1. spacecraft starts moving above planetary surface  |s| > Rp
    2. spacecraft's initial motion is directed towards the planet: v_y0 > 0
        
    Inputs:
        - s_x0, s_y0 (float): initial spacecraft position components in meters
        - v_x0, v_y0 (float): initial spacecraft velocity components in m/s
        - Rp (float): radius of the planet in meters
    
    Outputs:
        None
        
    Raises: 
        ValueError: if the initial position is invalid or if the velocity is not directed toward the planet.
    """
    
    # 행성으로부터의 길이 측정
    s_magnitude = math.sqrt(s_x0**2 + s_y0**2)

    # 행성으로부터의 길이가 행성의 반지름보다 큰지 아닌지 체크, 아닌 경우 ValueError
    if s_magnitude <= Rp:
        raise ValueError("Initial position must be above the planetary surface.")

    # 행성을 향하는 속도의 y 방향이 0보다 큰지 (양수) 체크, 아닌 경우 ValueError
    if v_y0 <= 0:
        raise ValueError("Initial velocity must be in the positive y-direction.")

    # 두 조건 다 통과하면(둘다 False), None을 return한다 
    return None

In [ ]:
def sc_vel_pos_change(a_x, a_y, v_x, v_y, time_step):
    """
    Computes the instantaneous change in position (ds_x, ds_y) 
    and velocity (dv_x, dv_y) of the spacecraft over a time-step (time_step).
    
    Inputs: a_x, a_y, v_x, v_y, time_step
    Outputs: ds_x, ds_y, dv_x, dv_y

    """
    # 속도 변화량
    dv_x = a_x * time_step
    dv_y = a_y * time_step

    # 위치 변화량
    ds_x = (v_x * time_step) + 0.5*a_x*(time_step**2)
    ds_y = (v_y * time_step) + 0.5*a_y*(time_step**2)

    return ds_x,ds_y,dv_x,dv_y